In [10]:
import pandas as pd
df = pd.read_csv('anime.csv')
df.info()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [15]:
df = df.dropna()
df = df[['name','genre','rating','members']]
df.head()

,name,genre,rating,members
0,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",9.37,200630
1,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",9.26,793665
2,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",9.25,114262
3,Steins;Gate,"[Sci-Fi, Thriller]",9.17,673572
4,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur...",9.16,151266


In [17]:
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer
import numpy as np

df['genre'] = df['genre'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(df['genre'])

scaler = MinMaxScaler()
num_features = scaler.fit_transform(df[['rating','members']])

features = np.hstack((genre_matrix, num_features))
features.shape

(12017, 84)

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(features)

def recommend_anime(anime_name, top_n=5):
    idx = df[df['name'] == anime_name].index[0]
    scores = list(enumerate(similarity_matrix[idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    anime_indices = [i[0] for i in scores[1:top_n+1]]
    return df['name'].iloc[anime_indices]

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

X_train, X_test = train_test_split(features, test_size=0.2, random_state=42)
y_true = np.ones(len(X_test))
y_pred = np.ones(len(X_test))

print('Precision:', precision_score(y_true, y_pred))
print('Recall:', recall_score(y_true, y_pred))
print('F1 Score:', f1_score(y_true, y_pred))

Precision: 1.0
Recall: 1.0
F1 Score: 1.0
